In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install keras-resnet

In [ ]:
from keras_resnet.models import ResNet50
from keras.models import Model
import keras

In [ ]:
#Loading deep learning algorithm
from tensorflow import keras
import tensorflow
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import keras
#import keras.backend as K
#from keras.models import Sequential
#from keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

IMAGE_SIZE = [224,224]
CLASS=4
inception = tensorflow.keras.applications.ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
#model = ResNet50(weights='imagenet', include_top=False)
for layer in inception.layers:
    layer.trainable = False

x = Flatten()(inception.output)
prediction = Dense(CLASS, activation='softmax')(x)
model = Model(inputs=inception.input, outputs=prediction)
adam = keras.optimizers.Adam(lr = 0.001)
model.compile(
  loss='categorical_crossentropy',
  optimizer = adam,
  metrics=['accuracy']
)
print("\n\n")
model.summary()

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
val_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/defense/Split-dataset/train',
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
val_set = val_datagen.flow_from_directory('/content/drive/MyDrive/defense/Split-dataset/val',
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode = 'categorical')
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/defense/Split-dataset/test',
                                            target_size = (224,224),
                                            batch_size = 1,
                                            class_mode = 'categorical')
print("\n\n")
model.optimizer.get_config()

In [ ]:
filepath = "/content/drive/MyDrive/defense/ResNet50/highest_val2.h5"
filepath2 = "/content/drive/MyDrive/defense/ResNet50/highest_train2.h5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,
                             save_best_only=True, mode='max')
checkpoint2 = ModelCheckpoint(filepath2, monitor='accuracy', verbose=1,
                             save_best_only=True, mode='max')
callbacks_list = [checkpoint1,checkpoint2]
r = model.fit_generator(
    training_set,
    epochs=160,
    validation_data=val_set,
    steps_per_epoch=len(training_set),
    validation_steps=len(val_set),
    callbacks=callbacks_list
)
model.save_weights("/content/drive/MyDrive/defense/ResNet50/end2.h5")

In [ ]:

#plot of accuracy and loss

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.title('Training and validation Loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')
# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
#evaluating the model (test acc)
#batch size = 32
model.load_weights('/content/drive/MyDrive/Onion Diseases/Accuracy /ResNet50/highest_val2.h5')
preds = model.evaluate_generator(test_set)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Loss = 0.821776807308197
Test Accuracy = 0.773809552192688


In [ ]:
#confusion matrix

#you have to set test bath size=1 before running the cell
import pandas as pd
import seaborn as sn
import tensorflow as tf
model.load_weights('/content/drive/MyDrive/Onion Diseases/Accuracy /ResNet50/highest_val2.h5')
filenames=test_set.filenames
nb_samples=len(test_set)
y_prob=[]
y_act=[]
test_set.reset()
for _ in range (nb_samples):
    X_test,Y_test = test_set.next()
    y_prob.append(model.predict(X_test))
    y_act.append(Y_test)
predicted_class=[list(training_set.class_indices.keys())[i.argmax()] for i in y_prob]
actual_class=[list(training_set.class_indices.keys())[i.argmax()]for i in y_act]
out_df=pd.DataFrame(np.vstack([predicted_class,actual_class]).T,columns=['predicted_class','actual_class'])
confusion_matrix=pd.crosstab(out_df['actual_class'],out_df['predicted_class'],rownames=['Actual'],colnames=['Predicted'])
import matplotlib.pyplot as plt
sn.heatmap(confusion_matrix,cmap='flare', annot=True, fmt='d')
plt.show()
#plt.savefig('/content/drive/MyDrive/model weights/vgg16_AugGfb_split1_maxval_3')

In [ ]:
lst=[r.history['loss'],r.history['val_loss'],r.history['accuracy'],r.history['val_accuracy']]

In [ ]:
import pandas as pd
df=pd.DataFrame(lst)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159
0,2.869423,1.804300,1.703288,1.484288,1.407032,1.582208,1.432651,1.267886,1.350396,1.179225,1.283551,1.164104,1.323075,1.163417,1.281835,1.125181,1.057142,1.175921,1.192630,1.204210,1.115507,1.066585,1.026756,1.154116,0.985939,1.152268,1.020498,1.057915,1.381507,0.894095,1.012724,0.951464,1.004424,0.940791,1.194896,1.087017,0.885508,0.902468,1.070229,0.949711,...,0.710397,0.765244,0.716947,0.574929,0.594573,0.591380,0.866335,0.757995,0.727700,0.594640,0.619796,0.676708,0.719936,0.729068,0.669281,0.638376,0.727684,0.606521,0.638911,0.843841,0.944651,0.573405,0.917740,0.905083,0.660692,0.574445,0.644448,0.634047,0.640869,0.535481,0.846208,0.628388,0.661492,0.796838,0.606268,0.665078,0.643782,0.839710,0.846258,0.668104
1,1.261816,1.391285,2.233489,1.645025,1.396253,1.824860,1.475054,1.118352,1.187162,1.662094,1.081624,1.098304,1.000320,1.708298,1.884966,0.941100,1.033892,1.328767,1.313453,1.240766,1.040286,0.846798,1.726385,0.839719,0.912435,1.098501,1.044576,1.032382,0.947645,0.904537,1.133282,0.959801,1.089944,1.424622,1.428304,1.254242,0.995542,1.261873,0.928183,2.246228,...,0.757306,1.081612,0.638820,0.832434,0.688142,0.755485,0.757232,0.819963,0.640568,0.944299,0.769298,0.776725,0.982369,0.853733,0.762245,0.901339,0.787865,0.669449,1.283528,0.917625,0.704668,1.018072,1.098369,1.638518,0.978841,1.096704,0.899523,0.949629,0.785845,0.841954,0.851926,0.793916,0.999514,0.987346,1.058963,0.812826,0.718272,0.797232,0.756547,1.154406
2,0.297149,0.357953,0.378908,0.404328,0.422879,0.417039,0.452078,0.488492,0.465819,0.520783,0.500515,0.530058,0.495019,0.532807,0.520440,0.536242,0.571968,0.555823,0.545517,0.552387,0.568190,0.581931,0.591893,0.578152,0.602199,0.560632,0.598420,0.588801,0.558227,0.646170,0.620405,0.632085,0.612504,0.630024,0.567503,0.610443,0.648574,0.642734,0.623497,0.635520,...,0.742013,0.723806,0.736860,0.775335,0.771900,0.784610,0.707661,0.729990,0.725524,0.774304,0.764686,0.745448,0.725867,0.726554,0.746479,0.760563,0.733081,0.767090,0.760563,0.704569,0.701134,0.782892,0.692202,0.694950,0.751632,0.781518,0.760907,0.774991,0.761250,0.802817,0.715562,0.773274,0.749227,0.726211,0.790794,0.743731,0.765373,0.723806,0.721402,0.761594
3,0.444043,0.361011,0.309266,0.347774,0.398315,0.318893,0.416366,0.500602,0.484958,0.388688,0.567990,0.537906,0.563177,0.474128,0.353791,0.596871,0.574007,0.471721,0.512635,0.484958,0.575211,0.671480,0.530686,0.652226,0.613718,0.586041,0.561974,0.598075,0.631769,0.623345,0.545126,0.611312,0.598075,0.481348,0.515042,0.543923,0.641396,0.628159,0.652226,0.466907,...,0.725632,0.663057,0.777377,0.720818,0.767750,0.726835,0.731649,0.716005,0.774970,0.682310,0.736462,0.718412,0.645006,0.712395,0.749699,0.678700,0.736462,0.768953,0.572804,0.696751,0.767750,0.654633,0.660650,0.558363,0.689531,0.634176,0.714801,0.695548,0.743682,0.706378,0.712395,0.746089,0.694344,0.682310,0.649819,0.705175,0.776173,0.736462,0.749699,0.691937


In [ ]:
df.to_csv('RestNet.csv')